In [1]:
import torch
if torch.cuda.get_device_capability() < (7, 5):
  raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: print("OK")

OK


In [2]:
# %pip install bitsandbytes datasets accelerate loralib
# %pip install bitsandbytes==0.37.0 transformers datasets accelerate==0.18.0 loralib peft
# %pip install bitsandbytes transformers datasets accelerate loralib peft
%pip install bitsandbytes==0.37.0 transformers==4.27.4 datasets==2.7.0 accelerate==0.18.0 loralib==0.1.1 peft==0.3.0.dev0
# %pip install transformers peft
# %pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git


Note: you may need to restart the kernel to use updated packages.


#### Load model

In [3]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import torch.nn.functional as F
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

CACHE_DIR = '/media/tfsservices/DATA/NLP/cache/'
# MODEL_NAME = "EleutherAI/gpt-j-6B"
MODEL_NAME = "EleutherAI/gpt-neox-20B"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,        # bitsandbytes lib required (convert the loaded model into mixed-8bit quantized model.)
    device_map='auto',
    torch_dtype=torch.float16,
    cache_dir=CACHE_DIR)      # path to a directory in which a downloaded pretrained model
    # low_cpu_mem_usage=True,   # loads the model using ~1x model size CPU memory
    # offload_state_dict=True)  # temporarily offload the CPU state dict to the hard drive to avoid getting out of CPU RAM
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)
tokenizer.pad_token = tokenizer.eos_token



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 46/46 [00:27<00:00,  1.65it/s]


#### Test generation

In [4]:
def generate_marketing(max_length = 150):
    
    batch = tokenizer("Hi {FirstName} ", return_tensors='pt').to('cuda')

    # with torch.no_grad():
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, min_length=30, max_length=max_length, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    print('\n\n', tokenizer.decode(output_tokens[0].cpu().numpy()))

In [5]:
def generate(prompt: str, max_length = 150):
    
    batch = tokenizer(prompt, return_tensors='pt').to('cuda')

    # with torch.no_grad():
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, min_length=30, max_length=max_length, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    print('\n\n', tokenizer.decode(output_tokens[0].cpu().numpy()))

In [6]:
generate_marketing()

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:220: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  attn_scores = torch.where(causal_mask, attn_scores, mask_value)




 Hi {FirstName} 
Hi {LastName}, I am {gender}.

and the corresponding template would look like this:
Hi <#=FirstName#> {Name} I am <#=Gender#>.

<|endoftext|>


#### Post-processing on the model

In [6]:
model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 6144)
    (layers): ModuleList(
      (0-43): 44 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=6144, out_features=18432, bias=True)
          (dense): Linear8bitLt(in_features=6144, out_features=6144, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=6144, out_features=24576, bias=True)
          (dense_4h_to_h): Linear8bitLt(in_features=24576, out_features=6144, bias=True)
          (act): FastGELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=6144, out_features=50432, bias=Fals

In [ ]:
# for module in model.modules():
#     if isinstance(module, bnb.nn.Linear8bitLt):
#         module.state.memory_efficient_backward = True

In [7]:
# for module in model.modules():
#     if isinstance(module, bnb.nn.Linear8bitLt):
#         module.state.memory_efficient_backward = True

for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
        param.data = param.data.to(torch.float32) # cast the small parameters (e.g. layernorm) to fp32 for stability

model.gradient_checkpointing_enable()  # reduce number of stored activations
# model.gpt_neox.project_in = lambda x: x.requires_grad_(True)
model.enable_input_require_grads()

# cast model outputs to float32 to unfuck the top-k sampler
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.embed_out = CastOutputToFloat(model.embed_out)

In [9]:
# # for module in model.modules():
# #     if isinstance(module, bnb.nn.Linear8bitLt):
# #         module.state.memory_efficient_backward = True

# for param in model.parameters():
#     param.requires_grad = False  # freeze the model - train adapters later
#     if param.ndim == 1:
#         param.data = param.data.to(torch.float32) # cast the small parameters (e.g. layernorm) to fp32 for stability

# model.gradient_checkpointing_enable()  # reduce number of stored activations
# # model.gpt_neox.project_in = lambda x: x.requires_grad_(True)
# model.enable_input_require_grads()

# # cast model outputs to float32 to unfuck the top-k sampler
# class CastOutputToFloat(nn.Sequential):
#     def forward(self, x): return super().forward(x).to(torch.float32)
# model.lm_head = CastOutputToFloat(model.lm_head)

In [8]:
model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 6144)
    (layers): ModuleList(
      (0-43): 44 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=6144, out_features=18432, bias=True)
          (dense): Linear8bitLt(in_features=6144, out_features=6144, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=6144, out_features=24576, bias=True)
          (dense_4h_to_h): Linear8bitLt(in_features=24576, out_features=6144, bias=True)
          (act): FastGELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): CastOutputToFloat(
    (0): Linear(in_features=6144, o

#### Apply LoRA

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=16, ## can be reduced to 8
    lora_alpha=32,
    # target_modules=["k_proj", "q_proj", "v_proj", "out_proj"], ## OK!
    # target_modules=["q_proj", "v_proj"],
    # target_modules=["k_proj", "q_proj", "v_proj", "out_proj", "lm_head"],
    # target_modules=["query_key_value", "dense"],
    # target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h", "embed_out" ],
    # target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h" ],
    target_modules=["query_key_value" ],
    lora_dropout=0.1, ## 0.05
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False # test,
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 17301504 || all params: 20571869184 || trainable%: 0.08410273196495162


In [11]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50432, 6144)
        (layers): ModuleList(
          (0-43): 44 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): MergedLinear8bitLt(
                in_features=6144, out_features=18432, bias=True
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=6144, out_features=32, bias=False)
                (lora_B): Conv1d(32, 12288, kernel_size=(1,), stride=(1,), groups=2, bias=False)
              )
              (dense): Linear8bitLt(in_features=6144, out_features=6144, bias=True)
            )
            (mlp): GPTNeoXMLP(
    

#### Test generation 

In [ ]:
# batch = tokenizer("Mark Zuckerberg is", return_tensors='pt')
# # test a single training step, make sure we get meaningful gradients
# with torch.cuda.amp.autocast():
#     out = model.forward(**batch)
#     out.logits.norm().backward()

# for module in model.modules():
#     if isinstance(module, LoraLinearModel):
#         assert module.adapter_B.grad is not None
#         assert module.adapter_B.grad.norm().item() > 0

# model.zero_grad(set_to_none=True)

In [10]:
# generate_marketing()

In [11]:
# generate_marketing()

#### Fine-tune model

In [9]:
# from datasets import load_dataset

# def prepare_query_and_response(raw):
#     # return f"Q: Generate {raw['tag']} \n A: {raw['text']} \n ###"
#     return f"{raw['text']} \n ###"

# dataset = load_dataset("csv", data_files="./datasets/wm.csv")
# tokenizer.pad_token = tokenizer.eos_token


# new_column = [prepare_query_and_response(r) for r in dataset["train"]]
# dataset["train"] = dataset["train"].add_column("text1", new_column)

# print(dataset['train'][0])

# dataset = dataset.map(lambda samples: tokenizer(samples['text1']), batched=True)

# ADAPTER_NAME = "adapters/gpt-neox-20B-wm"

Using custom data configuration default-dced1e05d608bdd0
Found cached dataset csv (/home/tfsservices/.cache/huggingface/datasets/csv/default-dced1e05d608bdd0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 921.02it/s]
Loading cached processed dataset at /home/tfsservices/.cache/huggingface/datasets/csv/default-dced1e05d608bdd0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c177dc266a56d917.arrow


{'text': 'Hi {FirstName},  \n\nHope all is well. I am {SenderName}, the trader from CAE that we received your web inquiry for \n{Listing}   \nI am checking the availability, meanwhile, please answer the following questions so that I may better assist you:   \nWhat is the reason for the purchase? \nWhen do you need the system delivered?   \nDo you have an approved budget for this purchase? How much is your budget?    \nCan you purchase the machine as-it-is since we do not provide warranty? \n What information do you require in order to make a decision? \n Do you have other machines for sale or purchase?    \nThanks.   \n\n{SendersSignature}', 'tag': 'Purchase Inquiry', 'text1': 'Hi {FirstName},  \n\nHope all is well. I am {SenderName}, the trader from CAE that we received your web inquiry for \n{Listing}   \nI am checking the availability, meanwhile, please answer the following questions so that I may better assist you:   \nWhat is the reason for the purchase? \nWhen do you need the sys

In [10]:
from datasets import load_dataset

def prepare_query_and_response(raw):
    # return f"Q: Generate {raw['tag']} \n A: {raw['text']} \n ###"
    return f"{raw['text']} \n ###"

dataset = load_dataset("csv", data_files="./datasets/armm.csv")
tokenizer.pad_token = tokenizer.eos_token


new_column = [prepare_query_and_response(r) for r in dataset["train"]]
dataset["train"] = dataset["train"].add_column("text1", new_column)

print(dataset['train'][0])

dataset = dataset.map(lambda samples: tokenizer(samples['text1']), batched=True)

ADAPTER_NAME = "adapters/gpt-neox-20B-armm"

Using custom data configuration default-6cdd53148b86b6af
Found cached dataset csv (/home/tfsservices/.cache/huggingface/datasets/csv/default-6cdd53148b86b6af/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 806.75it/s]
Loading cached processed dataset at /home/tfsservices/.cache/huggingface/datasets/csv/default-6cdd53148b86b6af/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-59c59dc0b0f179c1.arrow


{'text': 'Dear {FirstName}\n\nI have received a premium budget to purchase a {MakeModel} in the coming month. Do you have any old units sitting idle that you can consider selling? Their budget has been approved but they only accept offers after reviewing the configs details and photos.  My client ideally is looking for a tool that is in a fully working condition, but part systems can be considered as well.\n\n\n\nPlease contact me if you have such a tool or one similar. I would be happy to help you.\n\n\n\nBest regards,\n\n{SendersSignature}', 'tag': 'Requirement', 'text1': 'Dear {FirstName}\n\nI have received a premium budget to purchase a {MakeModel} in the coming month. Do you have any old units sitting idle that you can consider selling? Their budget has been approved but they only accept offers after reviewing the configs details and photos.  My client ideally is looking for a tool that is in a fully working condition, but part systems can be considered as well.\n\n\n\nPlease cont

In [11]:
import transformers

trainer = transformers.Trainer(
    model=model, 
    train_dataset=dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=10, 
        # max_steps=20, 
        max_steps=50, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=5, 
        output_dir='outputs_marketing'),
        
        # save_steps=100),
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.p

Step,Training Loss
5,3.117700
10,2.854100
15,2.349300
20,2.255400
25,2.035900
30,1.968900
35,1.893300
40,1.870600
45,1.827700


TrainOutput(global_step=50, training_loss=2.1966771697998047, metrics={'train_runtime': 562.6566, 'train_samples_per_second': 1.422, 'train_steps_per_second': 0.089, 'total_flos': 1.5092323950993408e+16, 'train_loss': 2.1966771697998047, 'epoch': 7.69})

#### Save LoRA adapter

In [14]:
# ADAPTER_NAME = "lora_adapters-6.7b"
# ADAPTER_NAME = "lora_adapters-13b"
# ADAPTER_NAME = "adapters/gpt-j-6B-armm"
# ADAPTER_NAME = "adapters/gpt-j-6B-wm"


model.save_pretrained(ADAPTER_NAME)

#### Test generation

In [12]:
generate_marketing()

/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")




 Hi {FirstName} 



We are currently looking for a new {MakeModel} 



The price of this {MakeModel} is {Price}

You can come see this {MakeModel} at once upon approval, we are currently accepting new offers.



{Listing}



Hope to hear from you soon.



Best regards,



{SendersSignature} 
 ### 
 ### 
 ### 

 ### 
 ### 
 ### 
 ###
 ### 
 ### 
 ###
 
 ### 
 ### 
 ### 
 ### 
 ###
 ### 
 ### 
 ###
 ###
 ###
 ### 
 ###


In [13]:
generate_marketing(max_length=250)



 Hi {FirstName} 



Hope this mail finds you well. I am reaching out for a second time on this equipment.  My client wants to proceed with the purchase and would like to review the units by the end of this week. Please let me know if you still have this piece available.



As you know, my client is a leading producer of {MakeModel} parts and they are looking at the availability of this unit currently listed for {Price} and also {MakeModel} units of comparable specification and age. 

Thank you.

{SendersSignature} 
 ### 
 ### 
 ### 
 
 ### 
 ### 
 ###
 ### 
 ### {FirstName} 
 ### 
 ### 
 ### 
 ### 
 ### 
 ###


 ### 
 ### 
 ### 
 ### 
 ### ### 
 ### 
 ###
 ### 
 ### 
 ### ### {SendersSignature} 
 ### 
 ### ### 
 ### 
 ### 
 ### 
 ###


 ### 

### ### 
 ### 

### 
 ### ### 
 ### 


In [17]:
# generate(prompt="Hi {FirstName} ", max_length=250)
generate(prompt="Q: Generate Listing \n A: ", max_length=250)



 Q: Generate Listing 
 A: 
How do you want me to prepare this request to get you a response? We will try to give you as much details as is possible to help you review it. Please let me know if I can assist you with any additional questions or concerns. 
B: 
Sure: thank you for reaching out, and please let me know if I can assist with any information or details for you to help in the sale of  this equipment?  What do you mean by "condition is as-is?" 
  Do you prefer photos or videos? 
  What's the highest price you are willing to accept? 
Thanks, 
  {SenderSignature} 
 ### For more listing details, pictures and additional information see: {AsIsMakeModel} 
 ### Please reply directly to this email. I will respond to you within 24 hours. 
 ### If you are not the recipient of the email or would like to receive any additional photos and/FirstListing, please let me know so that I can provide you with the images and information you are seeking. 
 Best Regards, 
  SenderSendersSignature 
 A:

In [18]:
generate(prompt="Q: Generate Listing \n A: ", max_length=250)



 Q: Generate Listing 
 A: 
First, please take a look at the photos before placing an order.
Second, this is a new list, so the price is still negotiable, so please ask me. 
 Thank you.
{
Q: Are these real pictures? 
 I have a buyer looking for {BuyersModel} or something they can purchase, 
Is anyone selling? 
 They would like to get in touch with you by email. 
If you hear about anything similar please also let me know. 
 Thanks, 
 {FirstName}, 
 {SendersSignature} 
  
 Q: Thank you for your message, 
  
  Did you find something? If so, could you please forward these photos to this buyer? 
    {PicsSender} 
  
  Also, feel free to pass along any other information you have on these products if it's not included in the photos. 
 ### ### ### 
MakeModel 
  
### ### ### ### ### 
 {BuyersSignature} 
 ### ### ### ### ### 
 Tel: 
  {SendersSignature} 
 
 ### ### 


In [13]:
generate(prompt="Dear {FirstName} ", max_length=250)



 Dear {FirstName} 
We are happy to receive your message regarding CAE/Sale. Thank you for sharing the details. 
Please note that we are an internationally active seller specializing in industrial equipment and machinery. So we have been able to support many global and local CAE/Sale customers.
Our website is www.cae.com. You (or your clients) may also find photos and more technical specifications there.
We encourage you to visit us on site in case you could visit or check the equipment you have in mind.
Please be free to contact us on WhatsApp +86 14-58330885 (WechatWechat) if you need to further clarify your requirements. 
Kind Regards 
{SendersSignature} 
 ### CAE CAE has been providing professional services for the international marketplace for over 40 years. Our network of experts and professionals provide in-depth assessment, analysis and support for new purchases and sale of used equipment from all around the world.
### ### 
 ### CAE ### 
 ### Sale ### ### 
 ###  
### 
 ### CAE

In [20]:
generate(prompt="Hi {FirstName} ", max_length=250)



 Hi {FirstName} 
Thank you for your reply.  It is a big challenge to find those that still operate like yours; mostly sold or already upgraded to a bigger format or a more specialized model.
Please advise what the price is for the unit you have available at the moment as I would like to check it myself with my colleagues in the USA. 
In addition, would like to send you some pictures and our technical sheets as well.
If we agree on a price I will ask our clients to purchase it, let me know the best way we can communicate. 
Thanks, 
{SendersSignature} 
  {FirstName} and 
{SenderSignature} 
 {Model} 
{SendersSignature} 
 ### 
 This email was received from the following system:  {SenderSignature} {Model} 
 A photo of the system is attached as well. 
{SenderSignature} from CAE 
 ### CAE Systems and Services  {Model} CAE-Senders 
 The CAE Systems and Services department provides a variety of equipment and services to the electrical and automation industries. CAE can provide you with the be

In [22]:
# generate(prompt="Requirement: Hi {FirstName} ", max_length=200)
# generate(prompt="Requirement: ", max_length=200)
generate(prompt="Hi {FirstName} ", max_length=250)



 Hi {FirstName} 
Thank you for reaching out. 
What do you need to see the unit to determine a price for? 
I have a few pieces for sale right now. 
Let me know about the specific seller you would like to talk with?  
 {SenderSignature} 
{SenderSignature} 
 ###Sender Information### 
 FirstName /SendersSenders{SenderSignature} ###EndSender### 
 ###SenderSignature### ###Sender### 
  
{ListName} 
 ###SenderComments### 
 ###SenderSenders### 
 {Listing} 
 ###To### 
  
### 
 ###SenderSignature### ###Sender### 
  
###EndSender### 
{SenderSignature} ###Sender### 
 ###SenderSignature### 
  
###Listing### 
  
{Listing} 
 ###To### 
  
### 
 ###SenderComments### 
 ###SenderSignature### 
  
###EndSender###<|endoftext|>


In [44]:
generate(prompt="Listing: Hi", max_length=200)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




 Listing: Hi {FirstName} -

I hope everything is going great on your side. Would you be interested in a {MakeModel} {Category} system? We have a complete unit with 3 chambers and one TM. TM is not for sale separately. If you are interested, please contact me for pictures. 

{Listing}


{SendersSignature}
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ### 
 ###
 ### 
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###
 ###



In [20]:
# generate(prompt="Inquiry: Hi {FirstName} ", max_length=200)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




 Inquiry: Hi {FirstName} 
This is {Sender} with CAE. 
Thank you for considering selling {MakeModel}. 
We received your last inquiry regarding {MakeModel}. 
Now would you be able to send over the photo of this system and configurations, along with the price you're seeking? 
Meanwhile, if there's any other system that I can help inquiring, please let me know. 
Thanks. 
{SenderSignature}
{SendersSignature}
//Signature
Inventory (click for full listing): 
{PicsListing}
Sending to office. 
Please let me know if you have any questions. 
Best regards, 
{SendersSignature}
Inquiry: I've received your inquiry. 
If you have any system that's still available for purchase, please let my know. 
Terms: 

